Тест

In [ ]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka import EmotionRecognizer

In [ ]:
#загрузка датафрейма
import pandas as pd
df = pd.read_csv('C:/Users/kozak/OneDrive/Рабочий стол/итмо поступление/НИРМА/sloyka/sloyka/sample_data/raw/comments_grazhdanka_spb_kalininsky.csv', sep = ';', index_col=0)

In [ ]:
df

Определение эмоции, используя одну модель. По умолчанию стоит модель HuggingFaceModel.Text.Bert_Large, но можно использовать также HuggingFaceModel.Text.Bert_Tiny, HuggingFaceModel.Text.Bert_Base, HuggingFaceModel.Text.Bert_Tiny2

In [ ]:
recognizer = EmotionRecognizer()
df['emotion'] = df['text'].apply(recognizer.recognize_emotion)

In [ ]:
df

Возможно более точный способ. Берёт среднее из нескольких моделей и возвращает доминирующую эмоцию, если указано average=True, или просто самую максимальную вероятность эмоции из моделей, если verage=False. 

In [ ]:
#df=df.head(100)
# Создание экземпляра класса EmotionRecognizer
emotion_recognizer = EmotionRecognizer()

# Применение функции recognize_average_emotion_from_multiple_models ко всем строкам DataFrame. 
# При average=True подсчёт будет по среднему значению.
# При average=False подсчёт будет по максимальному значению.
df['emotion_average'] = emotion_recognizer.recognize_average_emotion_from_multiple_models(df, 'text', average=True)

In [ ]:
df